# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2020
topic: str = "QUÍMICA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [4]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. En cuanto a las características estructural...
3,"DNA, señale la respuesta correcta:"
4,
5,1. La forma A tiene 24 pares de bases por vuel...
6,coidal.
7,2. La estructura de la forma B es más corta y ...
8,que las formas A y Z.
9,"3. En la forma B, el desplazamiento por residu..."


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,3.0,38,2.0,75,4,112,3.0,149,1.0
1,2,2.0,39,2.0,76,1,113,NaN,150,2.0
2,3,1.0,40,4.0,77,2,114,1.0,151,2.0
3,4,3.0,41,NaN,78,3,115,4.0,152,2.0
4,5,4.0,42,4.0,79,4,116,1.0,153,4.0
5,6,3.0,43,1.0,80,1,117,3.0,154,2.0
6,7,1.0,44,2.0,81,2,118,2.0,155,4.0
7,8,3.0,45,4.0,82,3,119,2.0,156,4.0
8,9,4.0,46,1.0,83,3,120,1.0,157,2.0
9,10,4.0,47,4.0,84,4,121,3.0,158,3.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [6]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [7]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [8]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [9]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [10]:
raw_exam_df.head(10)

0    1. En cuanto a las características estructural...
1                   DNA, señale la respuesta correcta:
2    1. La forma A tiene 24 pares de bases por vuel...
3                                              coidal.
4    2. La estructura de la forma B es más corta y ...
5                                que las formas A y Z.
6    3. En la forma B, el desplazamiento por residu...
7                                          unos 3.4 A.
8    4. La forma Z consiste en una doble hélice dex...
9    2. La hemólisis afecta sensiblemente a las con...
Name: text, dtype: object

In [11]:
raw_exam_df.tail(10)

1594                                 4. Errores anómalos.
1595    185. ¿Cuál de las siguientes afirmaciones es l...
1596                            una reacción enzimática?:
1597    1. La enzima empuja a la derecha el equilibrio...
1598                                            reacción.
1599    2. La enzima cambia el equilibrio constante de la
1600                                            reacción.
1601       3. La enzima aumenta la velocidad de reacción.
1602    4. La enzima altera la diferencia de energía e...
1603                              reactantes y productos.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [12]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [13]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. En cuanto a las características estructural...
1    1. La forma A tiene 24 pares de bases por vuel...
2    2. La estructura de la forma B es más corta y ...
3    3. En la forma B, el desplazamiento por residu...
4    4. La forma Z consiste en una doble hélice dex...
5    2. La hemólisis afecta sensiblemente a las con...
6     1. Aumenta el potasio y bajan el sodio y la LDH.
7     2. Aumentan la LDH y el potasio y baja el sodio.
8           3. Aumentan el sodio, la LDH y el potasio.
9    4. Aumentan la LDH y bajan el sodio y el potasio.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [14]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 4
Number of rows not ending with '.', ':' or '?' = 12
[47, 136, 206, 556, 671, 672, 673, 763, 845, 846, 847, 848]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [15]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [16]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [18]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [206, 556, 672, 763,]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 47:
45                         10. Los ácidos grasos trans:
46    1. Constituyen la forma natural de los ácidos ...
47    2. Promueven un aumento en la concentración de...
48    3. Presentan una temperatura de fusión inferio...
49    4. Se generan mayoritariamente en procesos ind...
Name: text, dtype: object
Id to fix 136:
134    4. La subunidad F0 sintetiza ATP gracias a cam...
135                             28. El DNA recombinante:
136    1. Se obtiene mediante la formación de heterod...
137    2. Es DNA híbrido obtenido mediante recombinac...
138            3. Es DNA híbrido celular superenrollado.
Name: text, dtype: object
Id to fix 206:
204                           4. 2,3,4-Trimetilpentanal.
205    42. Todas las siguientes especies son bases de...
206                        1: CH3OH 2: H2S 3: NH3 4: CH4
207                                                1. 1.
208                                                2. 2.
Name: text, dtype: object
Id to fix 556:
554    3. La 

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [19]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 925
925


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [20]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [21]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [22]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [23]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [24]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. En cuanto a las características estructural...,1. La forma A tiene 24 pares de bases por vuel...,2. La estructura de la forma B es más corta y ...,"3. En la forma B, el desplazamiento por residu...",4. La forma Z consiste en una doble hélice dex...
1,2. La hemólisis afecta sensiblemente a las con...,1. Aumenta el potasio y bajan el sodio y la LDH.,2. Aumentan la LDH y el potasio y baja el sodio.,"3. Aumentan el sodio, la LDH y el potasio.",4. Aumentan la LDH y bajan el sodio y el potasio.
2,3. La mayor parte de los equivalente reductore...,1. La ruta de las pentosas fosfato.,2. La glucólisis.,3. El ciclo del ácido cítrico.,4. La gluconeogénesis.
3,4. Si la secuencia de un fragmento de una cade...,1. 5‘- TTACGGATGCCGAAT-3‘.,2. 5‘- AATGCCTACGGCTTA-3‘.,3. 5‘- TAAGCCGTAGGCATT-3‘.,4. 5‘- UAAGCCGUAGGCAUU-3‘.
4,5. ¿Qué enzima NO interviene en la ruta de las...,1. 6-fosfogluconolactonasa.,2. Glucosa 6-fosfato deshidrogenasa.,3. 6-fosfogluconato deshidrogenasa.,4. Fosfoglucosa isomerasa.
5,"6. En referencia a la Somatostatina, señale la...",1. Es un tetradecapéptido que inhibe la hormon...,2. Es una hormona hipotalámica.,3. Es una hormona que estimula la secreción de...,4. Es una hormona que inhibe la insulina pancr...
6,7. Una enzima que sigue una cinética clásica d...,1. 40 μM/min.,2. 50 μM/min.,3. 70 μM/min.,4. 100 μM/min.
7,8. ¿Cuál de las siguientes afirmaciones es cor...,1. Produce varias moléculas de ATP en cada vue...,2. Se gastan varias moléculas de oxígeno direc...,3. Sus intermediarios se pueden reponer por re...,4. El principal punto de control del ciclo es ...
8,9. El término proteoma se refiere a:,1. El conjunto de todos los genes codificantes...,2. Las modificaciones post-traduccionales que ...,3. El conjunto de proteínas presentes en los l...,4. El conjunto de proteínas producido por el g...
9,10. Los ácidos grasos trans:,1. Constituyen la forma natural de los ácidos ...,2. Promueven un aumento en la concentración de...,3. Presentan una temperatura de fusión inferio...,4. Se generan mayoritariamente en procesos ind...


In [25]:
exam_df_pivot.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4
175,176. El valor predictivo positivo de una prueb...,1. El cociente de verdaderos positivos dividid...,2. El cociente de verdaderos positivos dividid...,3. La sensibilidad de la prueba dividida por 1...,4. El cociente de verdaderos negativos dividid...
176,177. La anemia perniciosa es uno de los síntom...,1. Níquel.,2. Calcio.,3. Magnesio.,4. Cobalto.
177,178. ¿Qué afirmación es correcta?:,1. En las células se producen modos de transpo...,2. En las células se producen modos de transpo...,3. En las células se producen modos de transpo...,4. En las células se producen modos de transpo...
178,179. ¿Qué es la fotoconductividad?:,1. Es el cambio de color que presenta un sólid...,2. Es el incremento de conductividad debido al...,3. Es el incremento de conductividad debido al...,4. Es la disminución de conductividad debido a...
179,180. ¿Cuál de las siguientes es una enzima que...,1. Dopamina beta-hidroxilasa.,2. Tirosinasa.,3. Ureasa.,4. Glutatión peroxidasa.
180,181. Si queremos estudiar la exactitud de un m...,1. Test t de Student de comparación de una med...,2. Test t de Student de comparación de pares d...,3. Test F de Fisher para comparar una media ex...,4. Test Q de Dixon de comparación de pares de ...
181,182. ¿Cuál de las siguientes frases es verdade...,1. Tienen idénticas propiedades fisiológicas.,2. Tienen rotaciones específicas de signos opu...,3. Tienen idénticas propiedades químicas (por ...,4. Tienen diferentes propiedades físicas.
182,183. Las formas de hemiacetal de los carbohidr...,1. Efecto anomérico.,2. Mutarrotación.,3. Inversión óptica.,4. Racemización.
183,184. ¿Cuál de los errores que pueden producirs...,1. Falsos negativos.,2. Errores aleatorios.,3. Falsos positivos.,4. Errores anómalos.
184,185. ¿Cuál de las siguientes afirmaciones es l...,1. La enzima empuja a la derecha el equilibrio...,2. La enzima cambia el equilibrio constante de...,3. La enzima aumenta la velocidad de reacción.,4. La enzima altera la diferencia de energía e...


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [26]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [27]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [28]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [29]:
answers_df_clean.head(10)

,V,RC
0,1,3.0
1,2,2.0
2,3,1.0
3,4,3.0
4,5,4.0
5,6,3.0
6,7,1.0
7,8,3.0
8,9,4.0
9,10,4.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [30]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. En cuanto a las características estructural...,1. La forma A tiene 24 pares de bases por vuel...,2. La estructura de la forma B es más corta y ...,"3. En la forma B, el desplazamiento por residu...",4. La forma Z consiste en una doble hélice dex...,3.0
1,2. La hemólisis afecta sensiblemente a las con...,1. Aumenta el potasio y bajan el sodio y la LDH.,2. Aumentan la LDH y el potasio y baja el sodio.,"3. Aumentan el sodio, la LDH y el potasio.",4. Aumentan la LDH y bajan el sodio y el potasio.,2.0
2,3. La mayor parte de los equivalente reductore...,1. La ruta de las pentosas fosfato.,2. La glucólisis.,3. El ciclo del ácido cítrico.,4. La gluconeogénesis.,1.0
3,4. Si la secuencia de un fragmento de una cade...,1. 5‘- TTACGGATGCCGAAT-3‘.,2. 5‘- AATGCCTACGGCTTA-3‘.,3. 5‘- TAAGCCGTAGGCATT-3‘.,4. 5‘- UAAGCCGUAGGCAUU-3‘.,3.0
4,5. ¿Qué enzima NO interviene en la ruta de las...,1. 6-fosfogluconolactonasa.,2. Glucosa 6-fosfato deshidrogenasa.,3. 6-fosfogluconato deshidrogenasa.,4. Fosfoglucosa isomerasa.,4.0
5,"6. En referencia a la Somatostatina, señale la...",1. Es un tetradecapéptido que inhibe la hormon...,2. Es una hormona hipotalámica.,3. Es una hormona que estimula la secreción de...,4. Es una hormona que inhibe la insulina pancr...,3.0
6,7. Una enzima que sigue una cinética clásica d...,1. 40 μM/min.,2. 50 μM/min.,3. 70 μM/min.,4. 100 μM/min.,1.0
7,8. ¿Cuál de las siguientes afirmaciones es cor...,1. Produce varias moléculas de ATP en cada vue...,2. Se gastan varias moléculas de oxígeno direc...,3. Sus intermediarios se pueden reponer por re...,4. El principal punto de control del ciclo es ...,3.0
8,9. El término proteoma se refiere a:,1. El conjunto de todos los genes codificantes...,2. Las modificaciones post-traduccionales que ...,3. El conjunto de proteínas presentes en los l...,4. El conjunto de proteínas producido por el g...,4.0
9,10. Los ácidos grasos trans:,1. Constituyen la forma natural de los ácidos ...,2. Promueven un aumento en la concentración de...,3. Presentan una temperatura de fusión inferio...,4. Se generan mayoritariamente en procesos ind...,4.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [31]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [32]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [33]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [34]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. En cuanto a las características estructural...,1. La forma A tiene 24 pares de bases por vuel...,2. La estructura de la forma B es más corta y ...,"3. En la forma B, el desplazamiento por residu...",4. La forma Z consiste en una doble hélice dex...,3,2020
1,2. La hemólisis afecta sensiblemente a las con...,1. Aumenta el potasio y bajan el sodio y la LDH.,2. Aumentan la LDH y el potasio y baja el sodio.,"3. Aumentan el sodio, la LDH y el potasio.",4. Aumentan la LDH y bajan el sodio y el potasio.,2,2020
2,3. La mayor parte de los equivalente reductore...,1. La ruta de las pentosas fosfato.,2. La glucólisis.,3. El ciclo del ácido cítrico.,4. La gluconeogénesis.,1,2020
3,4. Si la secuencia de un fragmento de una cade...,1. 5‘- TTACGGATGCCGAAT-3‘.,2. 5‘- AATGCCTACGGCTTA-3‘.,3. 5‘- TAAGCCGTAGGCATT-3‘.,4. 5‘- UAAGCCGUAGGCAUU-3‘.,3,2020
4,5. ¿Qué enzima NO interviene en la ruta de las...,1. 6-fosfogluconolactonasa.,2. Glucosa 6-fosfato deshidrogenasa.,3. 6-fosfogluconato deshidrogenasa.,4. Fosfoglucosa isomerasa.,4,2020
5,"6. En referencia a la Somatostatina, señale la...",1. Es un tetradecapéptido que inhibe la hormon...,2. Es una hormona hipotalámica.,3. Es una hormona que estimula la secreción de...,4. Es una hormona que inhibe la insulina pancr...,3,2020
6,7. Una enzima que sigue una cinética clásica d...,1. 40 μM/min.,2. 50 μM/min.,3. 70 μM/min.,4. 100 μM/min.,1,2020
7,8. ¿Cuál de las siguientes afirmaciones es cor...,1. Produce varias moléculas de ATP en cada vue...,2. Se gastan varias moléculas de oxígeno direc...,3. Sus intermediarios se pueden reponer por re...,4. El principal punto de control del ciclo es ...,3,2020
8,9. El término proteoma se refiere a:,1. El conjunto de todos los genes codificantes...,2. Las modificaciones post-traduccionales que ...,3. El conjunto de proteínas presentes en los l...,4. El conjunto de proteínas producido por el g...,4,2020
9,10. Los ácidos grasos trans:,1. Constituyen la forma natural de los ácidos ...,2. Promueven un aumento en la concentración de...,3. Presentan una temperatura de fusión inferio...,4. Se generan mayoritariamente en procesos ind...,4,2020


In [35]:
processed_exam_df.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
175,176. El valor predictivo positivo de una prueb...,1. El cociente de verdaderos positivos dividid...,2. El cociente de verdaderos positivos dividid...,3. La sensibilidad de la prueba dividida por 1...,4. El cociente de verdaderos negativos dividid...,1,2020
176,177. La anemia perniciosa es uno de los síntom...,1. Níquel.,2. Calcio.,3. Magnesio.,4. Cobalto.,4,2020
177,178. ¿Qué afirmación es correcta?:,1. En las células se producen modos de transpo...,2. En las células se producen modos de transpo...,3. En las células se producen modos de transpo...,4. En las células se producen modos de transpo...,1,2020
178,179. ¿Qué es la fotoconductividad?:,1. Es el cambio de color que presenta un sólid...,2. Es el incremento de conductividad debido al...,3. Es el incremento de conductividad debido al...,4. Es la disminución de conductividad debido a...,2,2020
179,180. ¿Cuál de las siguientes es una enzima que...,1. Dopamina beta-hidroxilasa.,2. Tirosinasa.,3. Ureasa.,4. Glutatión peroxidasa.,4,2020
180,181. Si queremos estudiar la exactitud de un m...,1. Test t de Student de comparación de una med...,2. Test t de Student de comparación de pares d...,3. Test F de Fisher para comparar una media ex...,4. Test Q de Dixon de comparación de pares de ...,1,2020
181,182. ¿Cuál de las siguientes frases es verdade...,1. Tienen idénticas propiedades fisiológicas.,2. Tienen rotaciones específicas de signos opu...,3. Tienen idénticas propiedades químicas (por ...,4. Tienen diferentes propiedades físicas.,4,2020
182,183. Las formas de hemiacetal de los carbohidr...,1. Efecto anomérico.,2. Mutarrotación.,3. Inversión óptica.,4. Racemización.,2,2020
183,184. ¿Cuál de los errores que pueden producirs...,1. Falsos negativos.,2. Errores aleatorios.,3. Falsos positivos.,4. Errores anómalos.,1,2020
184,185. ¿Cuál de las siguientes afirmaciones es l...,1. La enzima empuja a la derecha el equilibrio...,2. La enzima cambia el equilibrio constante de...,3. La enzima aumenta la velocidad de reacción.,4. La enzima altera la diferencia de energía e...,3,2020


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [36]:
id_filter = [x for x in range(0, 33, 1)] + [179, 184]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 35
Number of rows in filtered df = 35


**2. Showing the filtered DataFrame**

Using head and tail

In [37]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. En cuanto a las características estructural...,1. La forma A tiene 24 pares de bases por vuel...,2. La estructura de la forma B es más corta y ...,"3. En la forma B, el desplazamiento por residu...",4. La forma Z consiste en una doble hélice dex...,3,2020
1,2. La hemólisis afecta sensiblemente a las con...,1. Aumenta el potasio y bajan el sodio y la LDH.,2. Aumentan la LDH y el potasio y baja el sodio.,"3. Aumentan el sodio, la LDH y el potasio.",4. Aumentan la LDH y bajan el sodio y el potasio.,2,2020
2,3. La mayor parte de los equivalente reductore...,1. La ruta de las pentosas fosfato.,2. La glucólisis.,3. El ciclo del ácido cítrico.,4. La gluconeogénesis.,1,2020
3,4. Si la secuencia de un fragmento de una cade...,1. 5‘- TTACGGATGCCGAAT-3‘.,2. 5‘- AATGCCTACGGCTTA-3‘.,3. 5‘- TAAGCCGTAGGCATT-3‘.,4. 5‘- UAAGCCGUAGGCAUU-3‘.,3,2020
4,5. ¿Qué enzima NO interviene en la ruta de las...,1. 6-fosfogluconolactonasa.,2. Glucosa 6-fosfato deshidrogenasa.,3. 6-fosfogluconato deshidrogenasa.,4. Fosfoglucosa isomerasa.,4,2020
5,"6. En referencia a la Somatostatina, señale la...",1. Es un tetradecapéptido que inhibe la hormon...,2. Es una hormona hipotalámica.,3. Es una hormona que estimula la secreción de...,4. Es una hormona que inhibe la insulina pancr...,3,2020
6,7. Una enzima que sigue una cinética clásica d...,1. 40 μM/min.,2. 50 μM/min.,3. 70 μM/min.,4. 100 μM/min.,1,2020
7,8. ¿Cuál de las siguientes afirmaciones es cor...,1. Produce varias moléculas de ATP en cada vue...,2. Se gastan varias moléculas de oxígeno direc...,3. Sus intermediarios se pueden reponer por re...,4. El principal punto de control del ciclo es ...,3,2020
8,9. El término proteoma se refiere a:,1. El conjunto de todos los genes codificantes...,2. Las modificaciones post-traduccionales que ...,3. El conjunto de proteínas presentes en los l...,4. El conjunto de proteínas producido por el g...,4,2020
9,10. Los ácidos grasos trans:,1. Constituyen la forma natural de los ácidos ...,2. Promueven un aumento en la concentración de...,3. Presentan una temperatura de fusión inferio...,4. Se generan mayoritariamente en procesos ind...,4,2020


In [38]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
25,26. ¿Qué oligoelemento ejerce una función impo...,1. Selenio.,2. Manganeso.,3. Zinc.,4. Cobre.,3,2020
26,27. ¿Cuál de las siguientes afirmaciones es co...,"1. La subunidad F1, orientada al espacio inter...",2. La subunidad F0 tiene semicanales que permi...,3. La subunidad F1 canaliza el paso de protone...,4. La subunidad F0 sintetiza ATP gracias a cam...,2,2020
27,28. El DNA recombinante:,1. Se obtiene mediante la formación de heterod...,2. Es DNA híbrido obtenido mediante recombinac...,3. Es DNA híbrido celular superenrollado.,4. Se obtiene utilizando endonucleasas de rest...,4,2020
28,"29. En los estudios enzimáticos, una gráfica s...",1. Cinética de Michaelis-Menten.,2. Unión cooperativa.,3. Inhibición competitiva.,4. Inhibición no competitiva.,2,2020
29,30. ¿Cuál de las siguientes afirmaciones es co...,1. Se sustentan por enlaces covalentes entre s...,2. El colesterol forma asociaciones estables c...,"3. Son simétricas, con una distribución de fos...",4. El porcentaje en peso de proteína no varía ...,2,2020
30,31. ¿En qué lugar se localiza la enzima glucoq...,1. Hígado.,2. Músculo esquelético.,3. Riñón.,4. Tejido adiposo.,1,2020
31,32. Los quilomicrones son:,1. Partículas pequeñas que contienen principal...,2. Partículas pequeñas pobres en triglicéridos...,3. Partículas grandes producidas por el intest...,4. Partículas grandes producidas por el hígado...,3,2020
32,33. La cromatina está compuesta por:,1. DNA y 4 tipos de histonas.,2. DNA y proteínas.,3. DNA de doble cadena y heteroduplex DNA-RNA.,4. Nucleosomas.,0,2020
33,180. ¿Cuál de las siguientes es una enzima que...,1. Dopamina beta-hidroxilasa.,2. Tirosinasa.,3. Ureasa.,4. Glutatión peroxidasa.,4,2020
34,185. ¿Cuál de las siguientes afirmaciones es l...,1. La enzima empuja a la derecha el equilibrio...,2. La enzima cambia el equilibrio constante de...,3. La enzima aumenta la velocidad de reacción.,4. La enzima altera la diferencia de energía e...,3,2020


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [39]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [40]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [41]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT id FROM questions WHERE question = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [42]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], n+1, option, n+1 == question[6]))
            bir_warehouse.commit()